In [ ]:
import numpy as np
import sep 

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.patches import Ellipse

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

In [ ]:
# Read image into standard 2-d numpy array
hdul = fits.open("../data/image.fits")
data = hdul[0].data
hdul.close()

In [ ]:
# Show the image and save as PNG
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()
plt.savefig("image_original.png")  # Save the figure as PNG
plt.show()

In [ ]:
# Measure a spatially varying background on the image
bkg = sep.Background(data)

# Get a "global" mean and noise of the image background:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
# Evaluate background as 2-d array, same size as original image
bkg_image = bkg.back()

# Show the background and save as PNG
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()
plt.savefig("background_image.png")  # Save the figure as PNG
plt.show()

In [ ]:
# Evaluate the background noise as 2-d array, same size as original image
bkg_rms = bkg.rms()

# Show the background noise and save as PNG
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()
plt.savefig("background_noise.png")  # Save the figure as PNG
plt.show()

In [ ]:
# Subtract the background
data_sub = data - bkg

Object Detection

In [ ]:
# Object Detection
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

# How many objects were detected
print(len(objects))

In [ ]:
# Plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

In [ ]:
# Plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

In [ ]:
# Save the figure with detected objects as PNG
plt.savefig("detected_objects.png")  # Save the figure as PNG
plt.show()

In [ ]:
# Available fields
print(objects.dtype.names)

flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
# Show the first 10 objects results
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))